# 0. Install Dependencies

In [1]:
!pip install tensorflow==2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2

# 1. Test Toy Environment with OpenAI Gym

In [3]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random
from gym import spaces
import os


In [4]:
# inpute one annotator correction in the fporm of a dictionary like -
# {'S': 'Genetic risk refers more to your chance of inheriting a disorder or disease .', 
# 'A_list': [{'indices': ['3', '4'], 'error_tag': 'Rloc-', 'correction': ''}]}
def get_correct(data):
    sent = data["S"].split()
    shift = 0
    for action in data["A_list"]:
        index = action["indices"]
        sent = sent[:index[0]+shift] + [action["correction"]] + sent[index[1]+shift:]
        if index[0] == index[1]: shift += 1  
    return " ".join(sent)

In [5]:
# string matching reward
def get_ied(instseq1, instseq2):
    m = len(instseq1)
    n = len(instseq2)
    if min(m,n) == 0: return 0
    cost_matrix = {}
    for i in range(m+1):
        for j in range(n+1):
            if min(i,j) == 0:
                cost_matrix[str(i)+'_'+str(j)] = max(i,j)
                continue
            cost = 1
            try:
                if instseq1[i-1] == (instseq2[j-1]):
                    cost = 0
            except:
                raise Exception('levenshtein calculation error')
            a = cost_matrix[str(i-1)+'_'+str(j)]+1
            b = cost_matrix[str(i)+'_'+str(j-1)]+1
            c = cost_matrix[str(i-1)+'_'+str(j-1)]+cost
            cost_matrix[str(i)+'_'+str(j)] = min(a, min(b,c))
    ed = float(cost_matrix[str(m)+'_'+str(n)])
    return 1 - (ed / max(m,n))

def print_sent(sent):
    table = []
    sent = sent.split()
    for i in range(len(sent)):
        print(sent[i]+"(" + str(i) + ")", end = " ")

In [6]:
""" datapoint 301
[{'S': 'One of the diseases is sickle cell trait .',
  'A_list': [{'indices': [2, 3],
    'error_tag': 'ArtOrDet',
    'correction': 'these'}],
  'G': 'One of these diseases is sickle cell trait .'},
 {'S': 'One of the diseases is sickle cell trait .',
  'A_list': [{'indices': [1, 2], 'error_tag': 'Prep', 'correction': ''},
   {'indices': [2, 3], 'error_tag': 'ArtOrDet', 'correction': 'such'},
   {'indices': [3, 4], 'error_tag': 'Nn', 'correction': 'disease'}],
  'G': 'One  such disease is sickle cell trait .'}, ..........]
"""

" datapoint 301\n[{'S': 'One of the diseases is sickle cell trait .',\n  'A_list': [{'indices': [2, 3],\n    'error_tag': 'ArtOrDet',\n    'correction': 'these'}],\n  'G': 'One of these diseases is sickle cell trait .'},\n {'S': 'One of the diseases is sickle cell trait .',\n  'A_list': [{'indices': [1, 2], 'error_tag': 'Prep', 'correction': ''},\n   {'indices': [2, 3], 'error_tag': 'ArtOrDet', 'correction': 'such'},\n   {'indices': [3, 4], 'error_tag': 'Nn', 'correction': 'disease'}],\n  'G': 'One  such disease is sickle cell trait .'}, ..........]\n"

In [7]:
error_tags = ['ArtOrDet', 'Mec', 'Nn', 'Npos', 'Others', 'Pform', 'Pref', 'Prep', 'Rloc-', 'SVA', 'Sfrag', 'Smod', 
              'Srun', 'Ssub', 'Trans', 'Um', 'V0', 'Vform', 'Vm', 'Vt', 'WOadv', 'WOinc', 'Wci', 'Wform', 'Wtone']
feedback_template = {}
standard_string = "missing or incorrect"
feedback_template = ['ArtOrDet' : "Article", 
                  'Mec' : "Punctuation / capitalization / spelling",
                  'Nn' : "Noun number incorrect",
                  'Npos' : "Possesive Noun",
                  'Pform' : "Pronoun Form",
                  'Pref' : "Pronoun reference",
                  'Prep' : "Preposition",
                  'Rloc-' : "Local Redundency",
                  'SVA' : "Subject-verb-agreement",
                  'Sfrag' : "Sentence fragmant",
                  'Smod' : "Dangling MOdifier",  #modifier could be misinterpreted as being associated with a word other than the one intended
                  'Srun' : "Runons / comma splice",  # fault is the use of a comma to join two independent clauses
                  'Ssub' : "Subordinate clause", # "I know that Bette is a dolphin" - here "that Bette is a dolphin" occurs as the complement of the verb "know" 
                  'Trans' : "Conjuctions",
                  'V0' : "Missing verb",
                  'Vform' : "Verb form",
                  'Vm' : "Verb modal",
                  'Vt' : "Verb tense",
                  'WOadv' : "Adverb/adjective position",
                  'Wci' : " Wrong collocation", # it went fair well -> fairly
                  'Wform' : "Word form"
                 ]


# generating feedback with respect to annotator 1
def generate_feedback(datapoint):
    return feedback_template[datapoint[0]["error_tag"]]

SyntaxError: invalid syntax (<ipython-input-7-72dd9856ec4f>, line 5)

In [8]:
file_path_10gec = "../Datasets/10gec_annotations/"
file = open(file_path_10gec + "A1.m2")

Sample_sent = []
Vocab = set()
error_tags = set()
max_len = 0

for line in file.readlines():
    if line[0] == "S" and len(line[2:].strip())>0:
        Sample_sent.append(line[2:].strip())
        Vocab.update(set(line[2:].strip().split()))
        max_len = max(max_len, len(line[2:].strip().split()))
    if line[0] == "A":
        words = line.split("|||")
        Vocab.update(set(words[2].split()))
        error_tags.update(set(words[1].split()))
Vocab = list(Vocab) + ["reflected"]
vocab_size = len(Vocab)

In [9]:
for e in sorted(list(error_tags)):
    print("'" + e + "'", end=", ")

'ArtOrDet', 'Mec', 'Nn', 'Npos', 'Others', 'Pform', 'Pref', 'Prep', 'Rloc-', 'SVA', 'Sfrag', 'Smod', 'Srun', 'Ssub', 'Trans', 'Um', 'V0', 'Vform', 'Vm', 'Vt', 'WOadv', 'WOinc', 'Wci', 'Wform', 'Wtone', 

In [10]:
# Extract ground truth correct sentences for grammatically incorect sentences
# Ex: S The opposite is also true .
#     A 5 6|||Mec|||:|||REQUIRED|||-NONE-|||0

dataset = {}
annotators = 10
for an in range(1, 11):
    file = "A" + str(an) + ".m2"
    file = open(file_path_10gec + file)
    current_S = ""
    current_A = []
    ex_ind = -1
    for line in file.readlines():
        if line[0] == "S":
            ex_ind += 1
            current_S = line[2:].strip()
        elif line[0] == "A":
            ans  = {}
            cells = line.strip().split("|||")
            ans["indices"] = [int(i) for i in cells[0].split()[1:]]  # A 5 6
            ans["error_tag"] = cells[1]
            ans["correction"] = cells[2]
            current_A.append(ans)
        elif line.strip() == "":  # empty line
            annot = {"S" : current_S, "A_list" : current_A}
            dataset[ex_ind] = [annot] if an==1 else dataset[ex_ind] + [annot]
            dataset[ex_ind][an-1]["G"] = get_correct(dataset[ex_ind][an-1])
            current_S = ""
            current_A = []
        
data_size = len(dataset.keys())

In [118]:
# # dump the json in a file
# import json
# import os
# with open('./10gec_jsons/data.json', 'w') as fh:
#     json.dump(dataset, fh, indent=2)


In [119]:
# vocab = ["", "I", "he", "she", "they", "am", "is", "are", "was", "were", "go", "going", "goes", "went", "market", 
#          "amusement-park", "to", "a", "an", "the"]
# vocab_size = len(vocab)
# max_len = 10
# Sample_sent = ["I is going to market.", "She am go to market.", "They are go to a amusement-park.", 
#                "He going to an market.", "I were going."]

In [140]:
def get_reward(sent, data_id, dataset):
    data = dataset[data_id]
    max_reward = 0
    for annotator in dataset[data_id]:
        G = annotator["G"]
        max_reward = max(max_reward, get_ied(sent.split(), G.split()))
    return max_reward             
    

In [145]:
s = 'People get certain diseases because of changes .'
get_reward(s, 3, dataset)

0.8888888888888888

In [12]:
class OurCustomEnv(Env):

    def __init__(self):

        self.sent = ""
        self.data_id = -1
        #self.sales_function = sales_function     #calculating sales based on spends
        self.high =np.array([vocab_size] * max_len, dtype=np.int32)
        self.low =np.array([0] * max_len, dtype=np.int32)
        
        #we create an observation space with predefined range
        self.observation_space = Box(low=self.low, high=self.high, dtype = np.float32)

        #similar to observation, we define action space 
        self.action_space = ["Replace", "Delete", "Add"]
        
    
    def step(self, action, arg1, arg2=0):
        
        if action == "Delete":
            sent1 = self.sent.split()
            self.sent = " ".join(sent1[:arg1] + sent1[arg1+1:])
        
        elif action == "Add":
            sent1 = self.sent.split()
            self.sent = " ".join(sent1[:arg1] + [Vocab[arg2]] + sent1[arg1:])
            
        elif action == "Replace":
            sent1 = self.sent.split()
            self.sent = " ".join(sent1[:arg1] + [Vocab[arg2]] + sent1[arg1+1:])
        
#         reward = 0.0
        reward = get_reward(self.sent, self.data_id, dataset)
        done = True                            #Condition for completion of episode
        info = {}        

        return self.sent, reward, done, info 

    def reset(self):
        self.data_id = random.randint(0, data_size)-1
        print(self.data_id)
        self.sent = dataset[self.data_id ][0]["S"]
        return self.sent
    
def act(action, ind1, string=""):
    if action == "Delete":
        return env.step("Delete", ind1)
    else:
        return env.step(action, ind1, Vocab.index(string))    

In [13]:
env = OurCustomEnv()

/Users/shreya/opt/anaconda3/lib/python3.8/site-packages/gym/spaces/box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [14]:
state = env.reset()

1247


In [15]:
print_sent(state)

Secondly(0) ,(1) without(2) the(3) truly(4) communication(5) in(6) the(7) real(8) life(9) ,(10) people(11) lose(12) the(13) ability(14) to(15) see(16) the(17) truth(18) and(19) false(20) .(21) 

In [16]:
state

'Secondly , without the truly communication in the real life , people lose the ability to see the truth and false .'

In [162]:
act1 = act("Replace", 3, "given")
act1

('In conclusion , given the unsafe online environment , spend too much time on internet may be the potential negative factors of using online social networks .',
 0.8571428571428572,
 True,
 {})

In [163]:
act2 = act("Replace", 9, "spending")
act2

('In conclusion , given the unsafe online environment , spending too much time on internet may be the potential negative factors of using online social networks .',
 0.8928571428571429,
 True,
 {})

In [166]:
print_sent(env.sent)

In(0) conclusion(1) ,(2) given(3) the(4) unsafe(5) online(6) environment(7) ,(8) spending(9) too(10) much(11) time(12) on(13) internet(14) may(15) be(16) the(17) potential(18) negative(19) factors(20) of(21) using(22) online(23) social(24) networks(25) .(26) 

In [167]:
act3 = act("Add", 17, "one")
act3

('In conclusion , given the unsafe online environment , spending too much time on internet may be one the potential negative factors of using online social networks .',
 0.8571428571428572,
 True,
 {})

In [168]:
act4 = act("Add", 18, "of")
act4

('In conclusion , given the unsafe online environment , spending too much time on internet may be one of the potential negative factors of using online social networks .',
 0.8275862068965517,
 True,
 {})

In [17]:
dataset[1247]

[{'S': 'Secondly , without the truly communication in the real life , people lose the ability to see the truth and false .',
  'A_list': [{'indices': [3, 4], 'error_tag': 'ArtOrDet', 'correction': ''},
   {'indices': [4, 5], 'error_tag': 'Rloc-', 'correction': ''},
   {'indices': [5, 6], 'error_tag': 'Vform', 'correction': 'communicating'},
   {'indices': [6, 6], 'error_tag': 'Others', 'correction': 'with people'},
   {'indices': [7, 8], 'error_tag': 'ArtOrDet', 'correction': ''},
   {'indices': [16, 17], 'error_tag': 'Wci', 'correction': 'judge'},
   {'indices': [17, 17],
    'error_tag': 'Others',
    'correction': 'if something is'},
   {'indices': [19, 20], 'error_tag': 'Trans', 'correction': 'or'},
   {'indices': [20, 21], 'error_tag': 'Wci', 'correction': 'a lie'}],
  'G': 'Secondly , without   communicating with people in  real life , people lose the ability to judge if something is the truth or a lie .'},
 {'S': 'Secondly , without the truly communication in the real life , peo